A script to reconstruct a MOF particle from a subset of 8 projections from HSH75-Pd-HAADF_hdr0.ali (this code works for .mrc and .ali files), which originally is a stack of 29 projections between -70° and 70°.

For an introduction on Python classes, see tutorial_classes.ipynb.
For an introduction on how neural networks works, see https://www.3blue1brown.com/topics/neural-networks.
For more details about the training procedure and the overall pytorch framework, see https://pytorch.org/tutorials/beginner/basics/intro.html.

For more details about a particular function / method, see the specific documentation in the code.

For more details about NN-FBP implementation, see the original article: Fast Tomographic Reconstruction from Limited Data Using Artificial Neural
Networks, D. M. Pelt and al., 2013 (https://ieeexplore.ieee.org/document/6607157).

In [1]:
### Imports ###
from nntomo.utilities import get_MSE_loss
from nntomo.network import nnfbp_training
from nntomo.nnfbp import DatasetNNFBP
from nntomo.projection_stack import ProjectionStack
from nntomo.volume import Volume

%load_ext autoreload
%autoreload 2

<pre>
The original 29 projections from STEM measurements. The         A subset of 8 projections: a comparison will be made between
SIRT reconstruction using all these projections will be         the NN-FBP and the SIRT reconstructions to evaluate the
used as a reference for the real volume of the particule.       reconstruction performance for very small subsets of projections.

<img src="data/gifs/particule_29proj.gif" width="300"/>                                  <img src="data/gifs/particule_8proj.gif" width="300"/>
</pre>

## Training set

Here, a stack of ellipses and its associated projections are used for the training dataset. The training dataset is used to modify the values of the weights and biases of the network, by comparing the predicted output of each input of the dataset and the real outputs. The projections are made with the ASTRA toolbox. The corresponding dataset is created and saved. See the pytorch tutorial for more information about the use of Dataset objects.

In [2]:
# Generation of the ellipses
ellipses_volume = Volume.stack_7ellipses(100, shape=1024, semi_axis_range=(20,200), padding=100)
ellipses_volume.save()

Generation of the ellipses: [████████████████████████████████████████████████████████████] 100/100 Est wait 00:0.0

Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\rand7ellipses1024.mrc.
 ID: rand7ellipses1024


In [3]:
# Creation of a volume object for the ellipses
ellipses_volume = Volume.retrieve('rand7ellipses1024')

# Creation of the projections (9 projections from -90° to 70°)
ellipses_projections = ProjectionStack.from_volume(ellipses_volume, 9, 'full')

# Creation of the dataset for the neural network training:
training_dataset = DatasetNNFBP(ellipses_projections, ellipses_volume)

# Saving of the dataset:
training_dataset.save()

Saving dataset...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\datasets_files\rand7ellipses1024-full9th_rand7ellipses1024_bin.pickle.
 ID: rand7ellipses1024-full9th_rand7ellipses1024_bin


<pre>
Each slice of the ellipses stack:                          The 9 projections taken around the axis perpendicular to the screen:

<img src="data/gifs/ellipses.gif" width="300"/>              <img src="data/gifs/ellipses_proj.gif" width="800"/>
</pre>

## Validation set

Here, random spheres are used for the validation set. The validation set is used to assess the performance of the network during the training phase on unknown data, and decide when to stop the training to avoid overfitting. The projections are also made with the ASTRA toolbox. The corresponding dataset is created and saved. See the pytorch tutorial for more information about the use of Dataset objects.

In [4]:
# Generation of the spheres
spheres = Volume.random_spheres(60, shape=1024, radius_range=(5,100), padding=50)
spheres.save()

Generation of the spheres: [████████████████████████████████████████████████████████████] 60/60 Est wait 00:0.00

Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\randspheres1024.mrc.
 ID: randspheres1024


In [5]:
# Creation of a volume object for the spheres
spheres = Volume.retrieve('randspheres1024')

# Creation of the projections (9 projections from -90° to 70°)
spheres_projections = ProjectionStack.from_volume(spheres, 9, 'full')

# Creation of the dataset for the neural network training:
validation_dataset = DatasetNNFBP(spheres_projections, spheres)

# Saving of the dataset:
validation_dataset.save()

Saving dataset...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\datasets_files\randspheres1024-full9th_randspheres1024_bin.pickle.
 ID: randspheres1024-full9th_randspheres1024_bin


<pre>
The random spheres:                                                  The 9 projections:

<img src="data/gifs/random_spheres.gif" width="300"/>                              <img src="data/gifs/randspheres_proj.gif" width="300"/>
</pre>

## Neural network training

Training of the neural network using the computed datasets. The network informations are automatically saved every 30s.

In [6]:
# Dataset retrieving using the ids
training_dataset = DatasetNNFBP.retrieve("rand7ellipses1024-full9th_rand7ellipses1024_bin")
validation_dataset = DatasetNNFBP.retrieve("randspheres1024-full9th_randspheres1024_bin")

# Number of hidden nodes in the network
Nh = 8

# Training of NN-FBP
network = nnfbp_training(training_dataset, validation_dataset, Nh, max_epoch=50)

Start of training.
Epoch 1 (n=0)
-------------------------------
Avg MSELoss(): 0.033581 

Epoch 2 (n=0)
-------------------------------
Avg MSELoss(): 0.015099 

Epoch 3 (n=0)
-------------------------------


KeyboardInterrupt: 

## Reconstuctions

Computation of the reconstruction of the particule with 8 projections, using the network, and the SIRT algorithm. A quantitative comparison between NN-FBP and SIRT can then be made by computing the MSE loss of the reconstruction using a SIRT reconstruction with the 29 projections as a reference.

In [5]:
# The 29 projections stack of the MOF particule
mof_projections_file = "data/projection_files/HSH75-Pd-HAADF_hdr0.ali"
mof_29proj = ProjectionStack.from_mrc_file(mof_projections_file, 'tem')

# The stack with a subset of 8 projections
mof_8proj = mof_29proj.get_proj_subset(8)

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\mrcfile\mrcinterpreter.py:206: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\mrcfile\mrcinterpreter.py:216: RuntimeWarning: Unrecognised machine stamp: 0x00 0x00 0x00 0x00
  warnings.warn(str(err), RuntimeWarning)


In [6]:
# NN-FBP reconstruction for the 8 projections stack
nn_reconstr_8proj = mof_8proj.get_NNFBP_reconstruction(network)
nn_reconstr_8proj.save()

Start of NN reconstruction.
Reconstruction part 1/2: [████████████████████████████████████████████████████████████] 8/8 Est wait 00:0.010

Reconstruction part 2/2: [████████████████████████████████████████████████████████████] 8/8 Est wait 00:0.090

Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\nn_rand7ellipses1024-full9th_rand7ellipses1024_bin_8h_HSH75-Pd-HAADF_hdr0-sub8.mrc. ID: nn_rand7ellipses1024-full9th_rand7ellipses1024_bin_8h_HSH75-Pd-HAADF_hdr0-sub8


In [7]:
# SIRT reconstructions for the 8 projections stack, with 150 iterations
sirt_reconstr_8proj = mof_8proj.get_SIRT_reconstruction(150)
sirt_reconstr_8proj.save()

Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_HSH75-Pd-HAADF_hdr0-sub8.mrc. ID: sirt150_HSH75-Pd-HAADF_hdr0-sub8


In [8]:
# SIRT reference reconstruction with 150 iterations
sirt_reconstr_29proj = mof_29proj.get_SIRT_reconstruction(150)
sirt_reconstr_29proj.save()

Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_HSH75-Pd-HAADF_hdr0.mrc. ID: sirt150_HSH75-Pd-HAADF_hdr0


<pre>
The reference reconstruction,                  NN-FBP reconstruction (8 projections):                SIRT reconstruction (8 projections):
SIRT with 29 projections:

<img src="data/gifs/particule_sirt29.gif" width="300"/>            <img src="data/gifs/particule_nn8.gif" width="300"/>            <img src="data/gifs/particule_sirt8.gif" width="300"/>
isosurface value: 190/255                          isosurface value: 245/255                          isosurface value: 170/255

</pre>

## Segmentation and MSE computation

In [9]:
# Segmentation: all voxel values are set to 0 or 1, depending on a segmentation value. This value is set arbitrarily, by looking at the shape of the
# volume in imod (isosurface value).
nn_reconstr_8proj = nn_reconstr_8proj.get_segmented_volume(245/255)
sirt_reconstr_8proj = sirt_reconstr_8proj.get_segmented_volume(170/255)
sirt_reconstr_29proj = sirt_reconstr_29proj.get_segmented_volume(190/255)

In [10]:
# MSE computation
print(f"NN-FBP MSE: {get_MSE_loss(sirt_reconstr_29proj, nn_reconstr_8proj)}")
print(f"SIRT MSE: {get_MSE_loss(sirt_reconstr_29proj, sirt_reconstr_8proj)}")

NN-FBP MSE: 0.0001929197460412979
SIRT MSE: 0.0002627996727824211


In [11]:
import torch
import cupy

torch.cuda.empty_cache()
print(torch.cuda.memory_allocated()/1024**2)
print(torch.cuda.memory_reserved()/1024**2)
print(cupy.get_default_memory_pool().used_bytes()/1024**2)
print(cupy.get_default_memory_pool().total_bytes()/1024**2)

8.125
4096.0
24.078125
36.0
